# Introduction to TensorFlow with Python

TensorFlow:
- Developed by Google Brain
- Open Source
- Data flow graphs
- Mainly used for neural networks and deep learning
- Can use the GPU to train much faster
- Numerous APIs to make building neural networks and optimizers easy
- deploy on CPUs or GPUs on desktops or servers

![TF dataflow graph](images/tensors_flowing.gif)

### Installing TensorFlow

- `pip install tensorflow` for the CPU version


- require GPU with CUDA support version 3.0 or higher
- install CUDA
- install CUDNN
- `pip install tensorflow-gpu`


- more information in the Confluence article and at [tensorflow.org](https://tensorflow.org)

In [ ]:
import tensorflow as tf

a = tf.constant(3.0)
b = tf.constant(5.0)
c = tf.multiply(a,b)

c

In [ ]:
sess = tf.Session()
sess.run(c)

**TensorFlow also has Placeholders and Variables**
- Placeholders are Tensors, which you can feed input to
- Variables are tensors with weight, which can be changed during the sessions. Optimizers change the variables.

**Loss functions and Optimizers:**
- Loss functions measure errors between tensors
- Optimizers, try to minimize the loss functions by updating all Variable tensors

**Learning rate:**
- parameter to give to the optimizer. Too high and you won't be able to get accurate results. Too low and and training will take way too long.

In [ ]:
x = tf.placeholder(tf.float32, shape=[])
a = tf.Variable(2.0)
y = tf.constant(6.0)

loss = tf.losses.mean_squared_error(
            labels=y, 
            predictions=tf.multiply(a, x))

train = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

sess.run(tf.global_variables_initializer())

In [ ]:
sess.run(train, feed_dict={x:2.00})
sess.run(a, feed_dict={x:2.0})

### Neural Networks

You can use the lowest level TF Core to create a Neural Network and this way you will have a lot of control on what's happening inside.

However for most purposes you can use higher-level APIs such as.
- `tf.layers` - For Dense, Convolutional and Pooling layers
- `tf.losses` - For loss functions
- `tf.train` - For Optimizers
- `tf.estimator` - Makes running training and evaluation loops easier, and provides dataset management utilities

In [ ]:
# Dataset 
# https://en.wikipedia.org/wiki/Iris_flower_data_set

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

data = load_iris()
features = data.data
labels = data.target.reshape((-1, 1))

labels

In [ ]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(sparse=False)
enc.fit_transform(labels)

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(
                                features, 
                                enc.transform(labels) )

x_size = train_x.shape[1]
y_size = train_y.shape[1]

X = tf.placeholder(tf.float32, shape=[None, x_size])
Y = tf.placeholder(tf.float32, shape=[None, y_size])

In [ ]:
# Neural Network Model
def hidden_layer(t_input, w_shape, activation=tf.nn.sigmoid):
    biases = tf.Variable(tf.random_normal(w_shape))
    weights = tf.Variable(tf.random_normal(w_shape))
    return activation(tf.add(biases, tf.matmul(t_input, weights)))

h_layer1 = hidden_layer(X, [x_size, 128])
h_layer2 = hidden_layer(h_layer1, [128, 128])
y_hat = hidden_layer(h_layer2, [128, y_size], tf.nn.softmax)

In [ ]:
h_layer1 = tf.layers.dense(X, 128, activation=tf.nn.sigmoid)
h_layer2 = tf.layers.dense(h_layer1, 128, activation=tf.nn.sigmoid)
y_hat = tf.layers.dense(h_layer2, y_size, activation=tf.nn.softmax)

In [ ]:
# Loss and Optimization
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
            labels=Y, logits=y_hat))

train_step = tf.train.GradientDescentOptimizer(0.005).minimize(loss)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
# Training Loop
for epoch in range(300):
    for i in range(train_x.shape[0]):
        sess.run(train_step, feed_dict={X: train_x[i:i+1], Y: train_y[i: i+1]})                                                 
        
    if (epoch % 10 == 0):
        correct_prediction = tf.equal(tf.argmax(y_hat, 1), tf.argmax(Y, 1))                                                      
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))                                                       
        print("Epoch %d: Accuraccy = %f, Loss = %f" % (                                                                          
                          epoch,                                                                                                   
                          sess.run(accuracy, feed_dict={X: test_x, Y: test_y}),                                                    
                          sess.run(loss, feed_dict={X: train_x, Y: train_y})))
    